# KNN User-User Collaborative Filtering

The idea is to find similar users based on their preferences or behavior and recommend items that similar users liked but the target user has not interacted with. If User A and User B have rated a lot of the same items similarly, User B might enjoy items that User A has rated highly but hasn’t yet discovered.

Based on [Introduction to Recommender System](https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada):

> Collaborative methods for recommender systems are methods that are **based solely on the past interactions** recorded between users and items in order to produce new recommendations. These interactions are stored in the so-called “user-item interactions matrix”. Then, the main idea that rules collaborative methods is that these past user-item interactions are sufficient to detect similar users and/or similar items and make predictions based on these estimated proximities.

> User-User CF is **applying KNN to find the neighbor users** with similar prior item preferences, and prediction of target item I for user U will be computed from the preferences of the neighbors.
Notice that, when computing similarity between users, the number of “common interactions” (how much items have already been considered by both users?) should be considered carefully! Indeed, most of the time, we want to avoid that someone that only have one interaction in common with our reference user could have a 100% match and be considered as being “closer” than someone having 100 common interactions and agreeing “only” on 98% of them. So, we consider that two users are similar if they have interacted with a lot of common items in the same way (similar rating, similar time hovering…).

In this notebook, we would take a very simple and straight-forward approach to implement KNN User-User CF. We formulate the problem as predicting the rating between user U and item I based on the rating records.

Specifically, we will:
- Collect input containing user-item rating
- Build user-user similarity matrix
- Measure similarities between users by using plain item rating vectors
- Predict the rating between user U and item I by getting all users who have rated I, identify N neighbors and weighted average their ratings based on how similar the users are to the user U

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.math_utils import sigmoid
from src.model import User2UserCollaborativeFiltering
from src.train_utils import map_indice
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "002-cf-u2u"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-26 22:58:56.894 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 002-cf-u2u...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "002-cf-u2u",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/002-cf-u2u",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(n_users, n_items):
    model = User2UserCollaborativeFiltering(n_users, n_items)
    return model

# Test implementation

In [5]:
# Mock data
user_indices = [0, 0, 1, 1, 2, 2, 2]
item_indices = [0, 1, 1, 2, 3, 1, 2]
ratings = [1, 4, 4, 5, 3, 2, 4]
n_users = len(set(user_indices))
n_items = len(set(item_indices))

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]

print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)

model = init_model(n_users, n_items)

users = [0, 1, 2]
items = [2, 2, 0]
predictions = model.predict(users, items)
print(predictions)

Mock User IDs: [0, 0, 1, 1, 2, 2, 2]
Mock Item IDs: [0, 1, 1, 2, 3, 1, 2]
Ratings: [1, 4, 4, 5, 3, 2, 4]
[0.5 0.5 0.5]


In [6]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.99031217 0.98201379 0.73105858]


#### 🧐 Go into details

In [7]:
model.user_item_matrix

array([[1., 4., 0., 0.],
       [0., 4., 5., 0.],
       [0., 2., 4., 3.]])

In [8]:
model.user_similarity

array([[0.        , 0.60604322, 0.36030188],
       [0.60604322, 0.        , 0.81202071],
       [0.36030188, 0.81202071, 0.        ]])

In [9]:
# Inspect one example
user = 0
item = 2

sim_scores = model.user_similarity[user]
print(f"{sim_scores=}")

sim_scores=array([0.        , 0.60604322, 0.36030188])


In [10]:
# Only consider users who have rated the item
user_ratings = model.user_item_matrix[:, item]
print(f"{user_ratings=}")
sim_scores = sim_scores[user_ratings != 0]
print(f"{sim_scores=}")
user_ratings = user_ratings[user_ratings != 0]
print(f"{user_ratings=}")

user_ratings=array([0., 5., 4.])
sim_scores=array([0.60604322, 0.36030188])
user_ratings=array([5., 4.])


In [11]:
# Weighted average of ratings
print(f"Weighted average: {np.dot(sim_scores, user_ratings)}")
print(f"Normalization factor: {np.sum(sim_scores)}")
print(f"Predicted rating: {np.dot(sim_scores, user_ratings) / np.sum(sim_scores)}")
print(
    f"Predicted rating - sigmoid: {sigmoid(np.dot(sim_scores, user_ratings) / np.sum(sim_scores))}"
)

Weighted average: 4.471423593469625
Normalization factor: 0.9663450945516922
Predicted rating: 4.627149885356445
Predicted rating - sigmoid: 0.9903121712214553


In [12]:
recommendations = model.recommend(val_user_indices, k=2)

Generating recommendations:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
recommendations

{'user_indice': [np.int64(0),
  np.int64(0),
  np.int64(1),
  np.int64(1),
  np.int64(2),
  np.int64(2)],
 'recommendation': [np.int64(2),
  np.int64(1),
  np.int64(2),
  np.int64(3),
  np.int64(2),
  np.int64(1)],
 'score': [np.float64(0.9903121712214553),
  np.float64(0.9628273118576526),
  np.float64(0.9820137900379085),
  np.float64(0.9525741268224334),
  np.float64(0.9933071490757153),
  np.float64(0.9820137900379085)]}

# Prep data

In [14]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [15]:
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))
n_users = len(unique_user_ids)
n_items = len(unique_item_ids)

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-26 22:58:58.170 | INFO     | __main__:<module>:8 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [16]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)

user_indices = [idm.get_user_index(user_id) for user_id in user_ids]
item_indices = [idm.get_item_index(item_id) for item_id in item_ids]
ratings = train_df[args.rating_col].values.tolist()

val_user_indices = [idm.get_user_index(user_id) for user_id in val_df[args.user_col]]
val_item_indices = [idm.get_item_index(item_id) for item_id in val_df[args.item_col]]
val_ratings = val_df[args.rating_col].values.tolist()

# Train

In [17]:
model = init_model(n_users, n_items)

#### Predict before train

In [18]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence
216,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B00W9DHUBS,0.0,1636157516787,17530,1851,Video Games,Star Wars Battlefront Outer Rim - Xbox One Dig...,[As part of the STAR WARS Battlefront Season P...,"[Video Games, Xbox One, Downloadable Content]",14.99,"[-1, -1, -1, -1, -1, 118, 2849, 2809, 1259, 863]"
423,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B0C6DH316S,4.0,1642700155462,17530,3059,Computers,Logitech G PRO X Wireless Lightspeed Gaming He...,[],"[Video Games, PC, Accessories, Headsets]",253.82,"[-1, -1, -1, 118, 2849, 2809, 1259, 863, 2699,..."
1554,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B001G6064W,0.0,1642700155462,17530,3097,Video Games,PROTOTYPE - Playstation 3,"[Product Description, You are Alex Mercer, the...","[Video Games, Legacy Systems, PlayStation Syst...",28.0,"[-1, -1, -1, 118, 2849, 2809, 1259, 863, 2699,..."
1722,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B004ZCPKDG,0.0,1642699950266,17530,4311,Video Games,Wii Sports Resort,"[Product Description, Wii Sports Resort, Amazo...","[Video Games, Legacy Systems, Nintendo Systems...",99.95,"[-1, -1, -1, -1, 118, 2849, 2809, 1259, 863, 2..."
1861,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B0BF5VRLHV,5.0,1636157516787,17530,2699,Computers,Logitech G PRO Mechanical Gaming Keyboard - Sh...,[],"[Video Games, PC, Accessories, Gaming Keyboards]",148.0,"[-1, -1, -1, -1, -1, 118, 2849, 2809, 1259, 863]"
1891,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B08N6NCR3Q,4.0,1642699950266,17530,3644,Video Games,Thrustmaster T 16000M SPACE SIM DUO STICK (PC),[The THRUSTMASTER T.16000M FCS Space Sim Duo c...,"[Video Games, PC, Accessories, Controllers, Fl...",119.51,"[-1, -1, -1, -1, 118, 2849, 2809, 1259, 863, 2..."


In [19]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])

2024-09-26 22:58:58.420 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AFUWPAK6VCGEL2OVIL2YGZNFQJZQ and parent_asin = B0C6DH316S


array([0.5])

#### Training loop

In [20]:
model.fit(user_indices, item_indices, ratings)

# Predict

In [21]:
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.predict([user_indice], [item_indice])

2024-09-26 22:59:07.009 | INFO     | __main__:<module>:1 - Test predicting before training with user_id = AFUWPAK6VCGEL2OVIL2YGZNFQJZQ and parent_asin = B0C6DH316S


array([0.5])

# Evaluate

## Ranking metrics

In [22]:
recommendations = model.recommend(val_user_indices, k=args.top_K)

Generating recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

In [23]:
recommendations_df = pd.DataFrame(recommendations).pipe(create_rec_df, idm)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,10525,3636,0.993307,1.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B073SQ2S3K
1,10525,1279,0.993307,2.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B01K1JV9KE
2,10525,2364,0.993307,3.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B0002ILS1K
3,10525,1273,0.993307,4.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B002BSA298
4,10525,3666,0.993307,5.0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B002JTX7HI
...,...,...,...,...,...,...
189795,14213,2566,0.993307,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B00KWEH61U
189796,14213,2572,0.993307,197.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B01NAUEFNR
189797,14213,2573,0.993307,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B08N7PZK77
189798,14213,2586,0.993307,199.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B0722GG88M


In [24]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
509,AFUQQLR2N2LY7XPE4VJ5YF3LDZVA,B07YN82X3B,5.0,1.0
250,AGJO7OFBOKRLDTSEL2HHSZSKDQ4Q,B07PZ8NZSZ,1.0,1.0
1549,AF4QBZD2EXOTKIOOH4BOC4HZDHYA,B08NRVRF3J,3.0,1.0
395,AH6YPZLRQH6OSGLGBVTGCNSF7JQQ,B0BVVTQ5JP,5.0,1.0
1325,AESEOKCWWKUG7YPP43J2CRWAXQIA,B09GM4283G,5.0,1.0
...,...,...,...,...
1365,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00LMRL00O,0.0,18.0
528,AFB6FYPPCN33UMUU5536IHXNOHCQ,B00Z9TM72Q,0.0,18.0
1670,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07NKN4VR4,0.0,18.0
311,AFB6FYPPCN33UMUU5536IHXNOHCQ,B002Z01QO2,0.0,19.0


In [25]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
101,16168.0,0.0,0.993307,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B00DBKSN8M,0,NaN
181,16168.0,3009.0,0.993307,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B07MVY7989,0,NaN
10,16168.0,2982.0,0.993307,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B000FUWCRY,0,NaN
68,16168.0,2983.0,0.993307,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B0056JPS84,0,NaN
193,16168.0,2984.0,0.993307,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B09MFBJJQJ,0,NaN
...,...,...,...,...,...,...,...,...
191520,1883.0,2574.0,0.993307,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B006JKASAC,0,NaN
191498,1883.0,749.0,0.993307,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0037M5W24,0,NaN
191614,1883.0,4018.0,0.993307,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B07CV6LH3V,0,NaN
191566,1883.0,3443.0,0.993307,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00WOK8YX4,0,NaN


In [26]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [27]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values

In [28]:
classifications = model.predict(val_user_indices, val_item_indices)

In [29]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,classification_proba,label
0,AEHCPMCLIMWHW7TEFRCOZN4MOFGA,B00V5V3E38,0.0,1633967679730,10525,4119,Video Games,Legend of Kay Anniversary - Nintendo Wii U,"[10 years after its initial release, Legend of...","[Video Games, Legacy Systems, Nintendo Systems...",None,"[-1, -1, -1, -1, 3766, 4102, 2872, 2810, 502, ...",0.982014,0
1,AGFPKXT34G5FGWARKXZC4GJTJQUQ,B01GY3651O,0.0,1630491281842,11724,2977,Video Games,XCOM 2 Deluxe Edition [Online Game Code],[The XCOM 2 Digital Deluxe Edition includes th...,"[Video Games, PC, Games]",None,"[594, 4578, 3289, 4664, 1057, 3845, 3010, 2691...",0.969248,0
2,AH4AOFTTDPHPAFAAVFMAF25H2LIQ,B09B14PJCG,0.0,1641748747823,7401,4682,Video Games,A Plague Tale: Innocence (XB1) - Xbox One,[Follow the grim tale of young Amicia and her ...,"[Video Games, Xbox One, Games]",59.99,"[4064, 24, 1441, 1805, 2619, 2254, 2274, 1454,...",0.988758,0
3,AEXTTZIJDNXIXQZFR5O7IJRXO3GA,B081243BT6,0.0,1637074638494,5074,3104,Cell Phones & Accessories,Orzly Carrying case for Nintendo Switch OLED a...,[],"[Video Games, Nintendo Switch, Accessories, Ca...",29.99,"[-1, -1, -1, -1, -1, 811, 2540, 313, 4394, 1846]",0.500000,0
4,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,1616,4127,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[1459, 1860, 3264, 569, 2143, 773, 4483, 296, ...",0.981448,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AFPSHZKKUL2YDGIDBQQUNRGE5MXQ,B07DD7QTBM,0.0,1641121394999,1672,2700,None,Just Dance 2019 - Xbox One Standard Edition,"[Dance to your own beat with Just Dance 2019, ...","[Video Games, Xbox One, Games]",12.3,"[-1, -1, -1, 2830, 1342, 1294, 1749, 2558, 385...",0.993307,0
1894,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,49,1887,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 3179, 1489, 2225, 3399, 3142, 4247, 3801,...",0.731059,1
1895,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,6619,2246,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2780, 3158, 130, 1164, 1030]",0.980681,1
1896,AEBTSECUK7ZEECNSRHQLMKO3E5VA,B002BSA388,0.0,1642567970979,18651,3724,Video Games,Super Mario Galaxy 2,"[Product Description, Launch into a new univer...","[Video Games, Legacy Systems, Nintendo Systems...",80.82,"[-1, -1, -1, -1, 2135, 2729, 14, 3639, 3118, 955]",0.993307,0


In [30]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# Clean up

In [31]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/26 23:13:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run 002-cf-u2u at: http://localhost:5003/#/experiments/3/runs/e2a9e61d0a1c426fa7ba95ca9f57f6a7.
2024/09/26 23:13:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/3.


# Appendix

## Model returning same score for every user-item in top 100